In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skopt import gp_minimize
from astropy.coordinates import SkyCoord
from astropy import units as u
import os
from astropy.cosmology import default_cosmology
from astropy import units as u
from astropy.coordinates import SkyCoord, Distance
from astropy.cosmology import Planck15
from astropy.cosmology import WMAP5, WMAP7
from sklearn import cluster, mixture, metrics
from sklearn.neighbors import kneighbors_graph
import warnings

Загружаем файл с галактиками из скоплений с N_tot>70, для которых есть фотометрические данные и красные смещения

In [3]:
path = './'
df= pd.read_csv(path+'list_z_photo.csv')

In [4]:
df.head()

,Name,ra,dec,z_gal,|dered_g-dered_r|,iGrID
0,1237650370488762520,195.544557,-2.495210,0.087696,0.88330,211
1,1237650370488828045,195.704581,-2.510970,0.086480,0.89793,211
2,1237650370488762557,195.589523,-2.501693,0.080822,0.88170,211
3,1237650370488762524,195.547180,-2.526123,0.085518,0.59039,211
4,1237650370488828042,195.720356,-2.520542,0.082964,0.91098,211


Создадим колонку с переведенными в мегапарсеки красными смещениями

In [5]:
df['z_Mpc']=Distance(unit=u.Mpc, z = df['z_gal'], cosmology = WMAP7)

In [6]:
df.head()

,Name,ra,dec,z_gal,|dered_g-dered_r|,iGrID,z_Mpc
0,1237650370488762520,195.544557,-2.495210,0.087696,0.88330,211,398.771958
1,1237650370488828045,195.704581,-2.510970,0.086480,0.89793,211,392.908529
2,1237650370488762557,195.589523,-2.501693,0.080822,0.88170,211,365.737425
3,1237650370488762524,195.547180,-2.526123,0.085518,0.59039,211,388.273498
4,1237650370488828042,195.720356,-2.520542,0.082964,0.91098,211,376.000434


In [7]:
train = df[['ra','dec', 'z_Mpc','|dered_g-dered_r|']]

In [8]:
bandwidth =  cluster.estimate_bandwidth(train, quantile=0.006)
connectivity = kneighbors_graph(train, n_neighbors=9, include_self=False)
connectivity = 0.5 * (connectivity + connectivity.T)
ms = cluster.MeanShift(bandwidth=bandwidth, bin_seeding=True)
k_means = cluster.MiniBatchKMeans(n_clusters=107)
dbscan = cluster.DBSCAN(eps=4.3)
affinity_propagation = cluster.AffinityPropagation(damping=0.7)
ward = cluster.AgglomerativeClustering(n_clusters=107, linkage='ward', connectivity=connectivity)
average_linkage = cluster.AgglomerativeClustering(linkage="average", affinity="cityblock",
        n_clusters=107, connectivity=connectivity)
single_linkage = cluster.AgglomerativeClustering(linkage="single", affinity="cityblock",
        n_clusters=107, connectivity=connectivity)
birch = cluster.Birch(n_clusters=107)
gmm = mixture.GaussianMixture(n_components=107, covariance_type='full')
clustering_algorithms = (
        ('MiniBatchKMeans', k_means),
        ('AffinityPropagation', affinity_propagation),
        ('MeanShift', ms),
        ('Ward', ward),
        ('AgglomerativeClustering_average', average_linkage),
        ('AgglomerativeClustering_single', single_linkage),
        ('DBSCAN', dbscan),
        ('Birch', birch),
        ('GaussianMixture', gmm)
    )

In [9]:
warnings.filterwarnings('ignore', category=UserWarning, append=True)
for name, algorithm in clustering_algorithms:
    y_pred = algorithm.fit_predict(train)
    print(name+' : fowlkes_mallows_score = %.3f, v_measure_score = %.3f' % 
          (round(metrics.fowlkes_mallows_score(df.iGrID, y_pred),3), 
           round(metrics.v_measure_score(df.iGrID, y_pred),3)))

MiniBatchKMeans : fowlkes_mallows_score = 0.505, v_measure_score = 0.842
AffinityPropagation : fowlkes_mallows_score = 0.446, v_measure_score = 0.809
MeanShift : fowlkes_mallows_score = 0.612, v_measure_score = 0.862
Ward : fowlkes_mallows_score = 0.581, v_measure_score = 0.881
AgglomerativeClustering_average : fowlkes_mallows_score = 0.716, v_measure_score = 0.911
AgglomerativeClustering_single : fowlkes_mallows_score = 0.774, v_measure_score = 0.932
DBSCAN : fowlkes_mallows_score = 0.801, v_measure_score = 0.932
Birch : fowlkes_mallows_score = 0.481, v_measure_score = 0.836
GaussianMixture : fowlkes_mallows_score = 0.712, v_measure_score = 0.930
